In [1]:
!pip install statsmodels patsy

In [2]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.genmod.bayes_mixed_glm import PoissonBayesMixedGLM
from tqdm import tqdm
from patsy import dmatrices
from patsy import dmatrix
from statsmodels.discrete.count_model import ZeroInflatedPoisson

In [3]:
hourly_averages = pd.read_csv('./hourly_averages.csv')

In [4]:
# Create array of category names as they appear in the detections data. See paper for details of each category.
categories = ['car', 'person', 'trotro', 'stall', 'truck', 'stove', 'motorcycle', 'vendor', 'lorry', 'umbrella', 'bus', 'trash', 'taxi', 'van', 'debris', 'loudspeaker', 'bowl', 'food', 'animal', 'bicycle']

# Column names in the data frame for the number of counts of each category type in an image.
count_cols = [cat+'_counts' for cat in categories]

super_count_cols = ['people'+'_counts', 'small_vehicles'+'_counts', 'two_wheelers'+'_counts', 'large_vehicles'+'_counts', 'refuse'+'_counts', 'market'+'_counts', 'animal'+'_counts']

all_count_cols = count_cols + super_count_cols

vehicle_categories = ['car', 'trotro', 'truck', 'motorcycle', 'lorry', 'bus', 'taxi', 'van', 'bicycle']

# Define super categories
super_categories = {
    'people': ['person', 'vendor'],
    'small_vehicles': ['car', 'taxi', 'truck'],
    'two_wheelers': ['bicycle', 'motorcycle'],
    'large_vehicles': ['trotro', 'van', 'lorry', 'bus'],
    'refuse': ['trash', 'debris'],
    'market': ['umbrella', 'stall', 'bowl', 'food'],
    'animal': ['animal']
}

In [5]:
len(hourly_averages['directory_pair'].unique())
len(hourly_averages[hourly_averages['site_id']=='N1W']['directory_pair'].unique())

508

In [ ]:
import statsmodels.api as sm
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

# Decide whether to include "week" as a variable or not
week_bool = True

site = 'N1W'

if site == None:
    filtered_data = hourly_averages.copy()
else:
    filtered_data = hourly_averages[hourly_averages['site_id']==site].copy()
# filtered_data = hourly_averages[hourly_averages['people_counts'] > 0].copy()

super_category='people'
# Step 2: Create the endogenous variable (response variable)
endog = filtered_data[super_category+'_counts'].astype(int)

# Step 3: Convert relevant columns to categorical
filtered_data['hour'] = filtered_data['hour'].astype('category')
filtered_data['day'] = filtered_data['day'].astype('category')
if week_bool == True:
    filtered_data['week'] = filtered_data['week'].astype('category')
if not (site == None):
    filtered_data['site_id'] = filtered_data['site_id'].astype('category')
filtered_data['year'] = filtered_data['year'].astype('category')
filtered_data['directory_pair'] = filtered_data['directory_pair'].astype('category')

# Step 4: One-hot encode 'hour', 'day', 'week', 'site_id', and 'year' for fixed effects
if week_bool == True:
    if site == None:
        exog_fixed = pd.get_dummies(filtered_data[['hour', 'day', 'week', 'site_id', 'year','directory_pair']], drop_first=True)
    else:
        exog_fixed = pd.get_dummies(filtered_data[['hour', 'day', 'week', 'year','directory_pair']], drop_first=True)
else:
    if site == None:
        exog_fixed = pd.get_dummies(filtered_data[['hour', 'day', 'site_id', 'year','directory_pair']], drop_first=True)
    else:
        exog_fixed = pd.get_dummies(filtered_data[['hour', 'day', 'year','directory_pair']], drop_first=True)

# Step 5: Add intercept
exog_fixed = sm.add_constant(exog_fixed)

# Step 6: Convert exog_fixed to float
exog_fixed = exog_fixed.astype(float)

# Step 7: Fit the GLM with a Negative Binomial family
glm_model = sm.GLM(endog, exog_fixed, family=sm.families.NegativeBinomial())
# glm_model = sm.GLM(endog, exog_fixed, family=sm.families.Poisson())
glm_result = glm_model.fit()

# Step 8: Display the results of the fixed effects
print(glm_result.summary())



In [ ]:
# import statsmodels.api as sm
# import pandas as pd
# import numpy as np
# import matplotlib.pyplot as plt

# # Function to calculate the confidence intervals and exponentiate the coefficients
# def calculate_effects_and_ci(glm_result, var):
#     coef = glm_result.params[var]
#     conf = glm_result.conf_int().loc[var]
#     lower, upper = conf
#     return np.exp(coef), np.exp(lower), np.exp(upper)

# # Function to plot effects for a given variable
# def plot_effects(glm_result, var, x_labels, title, ref_class_label):
#     effects = [calculate_effects_and_ci(glm_result, v) for v in var]
#     estimates, lower_bounds, upper_bounds = zip(*effects)
    
#     # Add the reference class at the start
#     estimates = [1] + list(estimates)
#     lower_bounds = [1] + list(lower_bounds)
#     upper_bounds = [1] + list(upper_bounds)
#     x_labels = [ref_class_label] + x_labels
    
#     plt.figure(figsize=(12, 6))
#     plt.errorbar(range(len(x_labels)), estimates, yerr=[np.array(estimates) - np.array(lower_bounds), np.array(upper_bounds) - np.array(estimates)], fmt='o', ecolor='gray', capsize=5)
#     plt.xticks(ticks=range(len(x_labels)), labels=x_labels, rotation=45)
#     plt.xlabel('Categories')
#     plt.ylabel('Multiplicative Effect on Counts')
#     plt.title(title)
#     plt.grid(True)
    
#     # Highlight the reference class differently
#     plt.scatter(0, 1, color='red', zorder=5, label='Reference Class')
#     plt.legend()
#     # if site == None:
#     #     plt.savefig('results/time_series/'+title.replace(' ','_')+'.png', bbox_inches='tight')
#     # else:
#     #     plt.savefig('results/time_series/'+site+'/'title.replace(' ','_')+'.png', bbox_inches='tight')
#     plt.show()

# # Variables to plot
# hour_vars = [col for col in exog_fixed.columns if 'hour' in col and col != 'const']
# day_vars = [col for col in exog_fixed.columns if 'day' in col and col != 'const']
# site_vars = [col for col in exog_fixed.columns if 'site_id' in col and col != 'const']
# year_vars = [col for col in exog_fixed.columns if 'year' in col and col != 'const']
# week_vars = [col for col in exog_fixed.columns if 'week' in col and col != 'const'] if week_bool else []

# # X-axis labels
# hour_labels = [f'Hour {i}' for i in range(1, 24)]
# day_labels = ['Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']  # Assuming 'Mon' is the reference category
# site_labels = [label.split('_')[2] for label in site_vars]
# year_labels = [label.split('_')[1] for label in year_vars]
# week_labels = [label.split('_')[1] for label in week_vars]

# # Ensure the number of labels matches the number of data points
# assert len(hour_labels) == len(hour_vars)
# assert len(day_labels) == len(day_vars)
# assert len(site_labels) == len(site_vars)
# assert len(year_labels) == len(year_vars)
# if week_bool:
#     assert len(week_labels) == len(week_vars)

# # Plot effects
# plot_effects(glm_result, hour_vars, hour_labels, 'Effect of Hour of Day on People Counts', 'Hour 0')
# plot_effects(glm_result, day_vars, day_labels, 'Effect of Day of Week on People Counts', 'Mon')
# if site == None:
#     plot_effects(glm_result, site_vars, site_labels, 'Effect of Site ID on People Counts', 'AD')
# plot_effects(glm_result, year_vars, year_labels, 'Effect of Year on People Counts', '2019')

# if week_bool:
#     plot_effects(glm_result, week_vars, week_labels, 'Effect of Week on People Counts', 'Week 1')


In [ ]:
import statsmodels.api as sm
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

# Placeholder for storing results for each site
site_results = {}

# Decide whether to include "week" as a variable or not
week_bool = True
site_bool = True  # Assume you want to analyze all sites

super_category='people'
# If site_bool is True, analyze each site separately
if site_bool:
    unique_sites = hourly_averages['site_id'].unique()
    for site in unique_sites:
        print(f"Processing site: {site}")
        
        filtered_data = hourly_averages[hourly_averages['site_id'] == site].copy()
        
        # Step 2: Create the endogenous variable (response variable)
        endog = filtered_data[super_category+'_counts'].astype(int)
        
        # Step 3: Convert relevant columns to categorical
        filtered_data['hour'] = filtered_data['hour'].astype('category')
        filtered_data['day'] = filtered_data['day'].astype('category')
        if week_bool:
            filtered_data['week'] = filtered_data['week'].astype('category')
        filtered_data['year'] = filtered_data['year'].astype('category')
        filtered_data['directory_pair'] = filtered_data['directory_pair'].astype('category')
        
        # Step 4: One-hot encode 'hour', 'day', 'week', 'year', and 'directory_pair' for fixed effects
        if week_bool:
            exog_fixed = pd.get_dummies(filtered_data[['hour', 'day', 'week', 'year', 'directory_pair']], drop_first=True)
        else:
            exog_fixed = pd.get_dummies(filtered_data[['hour', 'day', 'year', 'directory_pair']], drop_first=True)
        
        # Step 5: Add intercept
        exog_fixed = sm.add_constant(exog_fixed)
        
        # Step 6: Convert exog_fixed to float
        exog_fixed = exog_fixed.astype(float)
        
        # Check for NaNs or Infs before modeling
        if endog.isna().sum() > 0 or np.any(np.isnan(exog_fixed)) or np.any(np.isinf(exog_fixed)):
            print(f"Skipping site {site} due to NaNs or Infs.")
            continue
        
        # Step 7: Fit the GLM with a Negative Binomial family
        try:
            glm_model = sm.GLM(endog, exog_fixed, family=sm.families.NegativeBinomial())
            glm_result = glm_model.fit()

            # Store results
            site_results[site] = glm_result

            # Step 8: Display the results of the fixed effects for each site
            print(f"\nResults for site: {site}")
            print(glm_result.summary())

        except Exception as e:
            print(f"Error fitting model for site {site}: {e}")
            continue  # Skip to the next site

else:
    # If not analyzing per site, run the model for the whole dataset
    filtered_data = hourly_averages.copy()
    
    endog = filtered_data[super_category+'_counts'].astype(int)
    
    filtered_data['hour'] = filtered_data['hour'].astype('category')
    filtered_data['day'] = filtered_data['day'].astype('category')
    if week_bool == True:
        filtered_data['week'] = filtered_data['week'].astype('category')
    filtered_data['year'] = filtered_data['year'].astype('category')
    filtered_data['directory_pair'] = filtered_data['directory_pair'].astype('category')
    
    if week_bool == True:
        exog_fixed = pd.get_dummies(filtered_data[['hour', 'day', 'week', 'year','directory_pair']], drop_first=True)
    else:
        exog_fixed = pd.get_dummies(filtered_data[['hour', 'day', 'year','directory_pair']], drop_first=True)
    
    exog_fixed = sm.add_constant(exog_fixed)
    exog_fixed = exog_fixed.astype(float)
    
    try:
        glm_model = sm.GLM(endog, exog_fixed, family=sm.families.NegativeBinomial())
        glm_result = glm_model.fit()
        site_results['all_sites'] = glm_result

        print("\nResults for all sites:")
        print(glm_result.summary())

    except Exception as e:
        print(f"Error fitting model for all sites: {e}")


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Function to calculate the confidence intervals and exponentiate the coefficients
def calculate_effects_and_ci(glm_result, var):
    coef = glm_result.params[var]
    conf = glm_result.conf_int().loc[var]
    lower, upper = conf
    return np.exp(coef), np.exp(lower), np.exp(upper)

# Function to plot effects for all sites
def plot_effects_multiple_sites(site_results, var, x_labels, title, ref_class_label):
    plt.figure(figsize=(12, 6))
    
    for i, (site, result) in enumerate(site_results.items()):
        # if site != 'EL':
        if site != None:
            # Filter out variables that are not present in the current result
            available_vars = [v for v in var if v in result.params.index]
            if not available_vars:
                continue  # Skip this site if no variables are available
            
            effects = [calculate_effects_and_ci(result, v) for v in available_vars]
            estimates, lower_bounds, upper_bounds = zip(*effects)
            
            # Adjust x_labels to match the filtered variables
            filtered_labels = [x_labels[var.index(v)] for v in available_vars]
            
            # Add the reference class at the start
            estimates = [1] + list(estimates)
            lower_bounds = [1] + list(lower_bounds)
            upper_bounds = [1] + list(upper_bounds)
            x_labels_mod = [ref_class_label] + filtered_labels
            
            plt.errorbar(
                range(len(x_labels_mod)), 
                estimates, 
                yerr=[np.array(estimates) - np.array(lower_bounds), np.array(upper_bounds) - np.array(estimates)], 
                fmt=f'{["o", "s", "D", "^"][i % 4]}', 
                label=f'Site: {site}',
                capsize=5
            )
    
    plt.xticks(ticks=range(len(x_labels_mod)), labels=x_labels_mod, rotation=45)
    plt.xlabel('Categories')
    plt.ylabel('Multiplicative Effect on Counts')
    plt.title(title)
    plt.grid(True)
    plt.legend()
    plt.show()

# Variables to plot
hour_vars = [col for col in exog_fixed.columns if 'hour' in col and col != 'const']
day_vars = [col for col in exog_fixed.columns if 'day' in col and col != 'const']
year_vars = [col for col in exog_fixed.columns if 'year' in col and col != 'const']
week_vars = [col for col in exog_fixed.columns if 'week' in col and col != 'const'] if week_bool else []

# X-axis labels
hour_labels = [f'Hour {i}' for i in range(1, 24)]
day_labels = ['Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']  # Assuming 'Mon' is the reference category
year_labels = [label.split('_')[1] for label in year_vars]
week_labels = [label.split('_')[1] for label in week_vars]

# Ensure the number of labels matches the number of data points
assert len(hour_labels) == len(hour_vars)
assert len(day_labels) == len(day_vars)
assert len(year_labels) == len(year_vars)
if week_bool:
    assert len(week_labels) == len(week_vars)

# Plot effects for all sites
plot_effects_multiple_sites(site_results, hour_vars, hour_labels, 'Effect of Hour of Day on '+super_category+' Counts for All Sites', 'Hour 0')
plot_effects_multiple_sites(site_results, day_vars, day_labels, 'Effect of Day of Week on '+super_category+' Counts for All Sites', 'Mon')
plot_effects_multiple_sites(site_results, year_vars, year_labels, 'Effect of Year on '+super_category+' Counts for All Sites', '2019')

if week_bool:
    plot_effects_multiple_sites(site_results, week_vars, week_labels, 'Effect of Week on '+super_category+' Counts for All Sites', 'Week 1')


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Function to calculate the confidence intervals and exponentiate the coefficients
def calculate_effects_and_ci(glm_result, var):
    coef = glm_result.params[var]
    conf = glm_result.conf_int().loc[var]
    lower, upper = conf
    return np.exp(coef), np.exp(lower), np.exp(upper)

# Function to plot effects for a single site
def plot_effects_single_site(site, glm_result, var, x_labels, title, ref_class_label):
    available_vars = [v for v in var if v in glm_result.params.index]
    effects = [calculate_effects_and_ci(glm_result, v) for v in available_vars]
    
    # Filter x_labels based on available variables
    filtered_labels = [x_labels[var.index(v)] for v in available_vars]
    
    # Add the reference class at the start
    estimates = [1] + [e[0] for e in effects]
    lower_bounds = [1] + [e[1] for e in effects]
    upper_bounds = [1] + [e[2] for e in effects]
    filtered_labels = [ref_class_label] + filtered_labels
    
    plt.figure(figsize=(12, 6))
    plt.errorbar(
        range(len(filtered_labels)), 
        estimates, 
        yerr=[np.array(estimates) - np.array(lower_bounds), np.array(upper_bounds) - np.array(estimates)], 
        fmt='o', 
        ecolor='gray', 
        capsize=5
    )
    plt.xticks(ticks=range(len(filtered_labels)), labels=filtered_labels, rotation=45)
    plt.xlabel('Categories')
    plt.ylabel('Multiplicative Effect on Counts')
    plt.title(f"{title} for Site: {site}")
    plt.grid(True)
    plt.savefig('./results/time_series/'+site+'/'+super_category+'/'+title.replace(' ','_')+'.png', bbox_inches='tight')
    # plt.show()

# Variables to plot
hour_vars = [col for col in exog_fixed.columns if 'hour' in col and col != 'const']
day_vars = [col for col in exog_fixed.columns if 'day' in col and col != 'const']
year_vars = [col for col in exog_fixed.columns if 'year' in col and col != 'const']
week_vars = [col for col in exog_fixed.columns if 'week' in col and col != 'const'] if week_bool else []

# X-axis labels
hour_labels = [f'Hour {i}' for i in range(1, 24)]
day_labels = ['Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']  # Assuming 'Mon' is the reference category
year_labels = [label.split('_')[1] for label in year_vars]
week_labels = [label.split('_')[1] for label in week_vars]

# Ensure the number of labels matches the number of data points
assert len(hour_labels) == len(hour_vars)
assert len(day_labels) == len(day_vars)
assert len(year_labels) == len(year_vars)
if week_bool:
    assert len(week_labels) == len(week_vars)

# Plot effects for each site separately
for site, result in site_results.items():
    if site != 'EL':  # Skip the 'EL' site if necessary
        plot_effects_single_site(site, result, hour_vars, hour_labels, 'Effect of Hour of Day on '+super_category+' Counts', 'Hour 0')
        plot_effects_single_site(site, result, day_vars, day_labels, 'Effect of Day of Week on '+super_category+' Counts', 'Mon')
        plot_effects_single_site(site, result, year_vars, year_labels, 'Effect of Year on '+super_category+' Counts', '2019')
        
        if week_bool:
            plot_effects_single_site(site, result, week_vars, week_labels, 'Effect of Week on '+super_category+' Counts', 'Week 1')
